In [1]:
import numpy as np
import os
import pandas as pd
from scipy.optimize import minimize, Bounds

We load our example grid and extract some useful data.

In [2]:
input_path = os.path.join("..", "example", "input", "generation", "case118_l2rpn_wcci")
df = pd.read_csv(os.path.join(input_path, "prods_charac.csv"))


n = df.shape[0]
avg_pmaxs = df.groupby(["type"])["Pmax"].mean()
types = avg_pmaxs.index.to_numpy()
avg_pmaxs = avg_pmaxs.to_numpy()
capacity_factor = np.array([30, 95, 15, np.nan, 25])
average_load = 2800
target_energy_mix = np.array([9., 36., 17., 2., 36.])
target_pmax = 15000

In [3]:
def type2idx(t):
  result = np.where(types == t)
  return result[0][0]

In [4]:
x = np.ones(6+n) * 1000

for i, row in df.iterrows():

  x[i+6] = type2idx(row["type"])

In [5]:
def get_pmaxs(x):
  idx = np.round(x)
  pmaxs = np.zeros(types.shape)
  for i in range(types.shape[0]):
    mask = idx == i
    pmaxs[i] = avg_pmaxs[i] * mask.sum()
  return pmaxs

def get_apriori_energy_mix(pmaxs, capacity_factor, average_load):
  apriori_energy_mix = capacity_factor * pmaxs / average_load
  total = np.nansum(apriori_energy_mix)
  if total > 100:
    #print("WARNING: apriori energy mix w/o thermal exceeds average load!")
    apriori_energy_mix[0] -= (total - 100)
    apriori_energy_mix[3] = 0
  else:
    apriori_energy_mix[3] = 100 - total
  return apriori_energy_mix

def split_slack_vars(x):
  n = types.shape[0] + 1
  return x[:n], x[n:]

In [6]:
def objective(x):
  slacks, _ = split_slack_vars(x)
  return slacks.sum()

# x >= 0
def constraint_1(x):
  return x

# x =< 4
def constraint_2(x):
  _, x = split_slack_vars(x)
  return -x + 4

def constraint_3(x):
  slacks, x = split_slack_vars(x)
  slacks = slacks[:-1]
  pmaxs = get_pmaxs(x)
  apriori_energy_mix = get_apriori_energy_mix(pmaxs, capacity_factor, average_load)
  return apriori_energy_mix - target_energy_mix + slacks

def constraint_4(x):
  slacks, x = split_slack_vars(x)
  slacks = slacks[-1]
  pmaxs = get_pmaxs(x)
  return -pmaxs.sum() + target_pmax + slacks

  
constraints = [
  {"type": "ineq", "fun": constraint_1},
  {"type": "ineq", "fun": constraint_2},
  {"type": "ineq", "fun": constraint_3, "tol": 1e-4},
  {"type": "ineq", "fun": constraint_4, "tol": 1e2}
]

In [7]:
bounds = Bounds(0, np.inf)
options = {"maxiter": 500_000}
res = minimize(objective, x, constraints=constraints, method="cobyla", options=options)
res

     fun: 38.07303571428589
   maxcv: 6.661338147750939e-16
 message: 'Optimization terminated successfully.'
    nfev: 5531
  status: 1
 success: True
       x: array([-1.49928868e-19, -8.18885810e-21,  1.25016071e+01,  2.00000000e+00,
        2.35714286e+01, -8.60186547e-20,  3.66873317e+00,  1.68251312e+00,
        3.57112348e+00,  3.92804674e+00,  2.01830485e+00,  1.79827479e+00,
        1.16639305e+00,  2.21452078e+00,  3.99552530e+00,  2.43642466e+00,
        2.04734507e-01,  3.88250670e+00,  2.41849642e+00,  2.43631228e+00,
        3.82064291e+00,  3.06028179e+00,  3.86418491e+00,  1.51050275e+00,
        3.53754726e+00,  4.20036401e-01,  3.99943342e+00,  1.98604683e+00,
        2.08902452e+00,  1.72009002e+00,  3.59055625e+00,  2.80805530e+00,
        3.99794225e+00,  3.11094131e+00,  1.98697551e+00,  3.87319007e+00,
        3.98748947e+00,  3.62827402e+00,  3.98920064e+00,  1.73226521e+00,
        3.62053246e+00,  1.17019412e+00,  3.73381225e+00,  3.77333501e+00,
        1.001

In [8]:
x = res.x

In [9]:
constraint_1(x)

array([-1.49928868e-19, -8.18885810e-21,  1.25016071e+01,  2.00000000e+00,
        2.35714286e+01, -8.60186547e-20,  3.66873317e+00,  1.68251312e+00,
        3.57112348e+00,  3.92804674e+00,  2.01830485e+00,  1.79827479e+00,
        1.16639305e+00,  2.21452078e+00,  3.99552530e+00,  2.43642466e+00,
        2.04734507e-01,  3.88250670e+00,  2.41849642e+00,  2.43631228e+00,
        3.82064291e+00,  3.06028179e+00,  3.86418491e+00,  1.51050275e+00,
        3.53754726e+00,  4.20036401e-01,  3.99943342e+00,  1.98604683e+00,
        2.08902452e+00,  1.72009002e+00,  3.59055625e+00,  2.80805530e+00,
        3.99794225e+00,  3.11094131e+00,  1.98697551e+00,  3.87319007e+00,
        3.98748947e+00,  3.62827402e+00,  3.98920064e+00,  1.73226521e+00,
        3.62053246e+00,  1.17019412e+00,  3.73381225e+00,  3.77333501e+00,
        1.00149809e+00,  3.59362139e+00,  3.99959080e+00,  3.92928994e+00,
        3.35788999e-01,  2.27827466e+00,  1.59492061e-01,  4.49143944e-01,
        3.73748474e+00,  

In [10]:
constraint_2(x)

array([3.31266830e-01, 2.31748688e+00, 4.28876523e-01, 7.19532647e-02,
       1.98169515e+00, 2.20172521e+00, 2.83360695e+00, 1.78547922e+00,
       4.47470015e-03, 1.56357534e+00, 3.79526549e+00, 1.17493302e-01,
       1.58150358e+00, 1.56368772e+00, 1.79357093e-01, 9.39718213e-01,
       1.35815093e-01, 2.48949725e+00, 4.62452737e-01, 3.57996360e+00,
       5.66583803e-04, 2.01395317e+00, 1.91097548e+00, 2.27990998e+00,
       4.09443748e-01, 1.19194470e+00, 2.05774569e-03, 8.89058687e-01,
       2.01302449e+00, 1.26809927e-01, 1.25105268e-02, 3.71725983e-01,
       1.07993588e-02, 2.26773479e+00, 3.79467539e-01, 2.82980588e+00,
       2.66187752e-01, 2.26664987e-01, 2.99850191e+00, 4.06378614e-01,
       4.09197213e-04, 7.07100633e-02, 3.66421100e+00, 1.72172534e+00,
       3.84050794e+00, 3.55085606e+00, 2.62515257e-01, 2.05246665e+00,
       2.60835409e+00, 1.77562714e+00, 1.86587528e+00, 7.23083401e-02,
       8.65753354e-01, 3.70826918e+00, 2.47072232e+00, 3.98000163e-01,
      

In [11]:
constraint_3(x)

array([ 6.21589286e+00,  3.18571429e+01,  6.21724894e-14, -6.66133815e-16,
        1.17239551e-13])

In [12]:
constraint_4(x)

8704.663636363635

In [13]:
_, x = split_slack_vars(x)

In [14]:
def get_nb_power_plants(x):
  power_plants = {
      "hydro": 0,
      "nuclear": 0,
      "solar": 0,
      "thermal": 0,
      "wind": 0
    }
  idx = np.round(x)
  for i in idx:
    key = list(power_plants.keys())[int(i)]
    power_plants[key] += 1
  return power_plants

In [15]:
get_nb_power_plants(x)

{'hydro': 6, 'nuclear': 5, 'solar': 18, 'thermal': 4, 'wind': 29}

In [16]:
pmaxs = get_pmaxs(x)
em = get_apriori_energy_mix(pmaxs, capacity_factor, average_load)
error = np.abs(target_energy_mix - em).sum()
print(f"Target energy mix: {target_energy_mix}")
print(f"Actual energy mix: {em}")
print(f"Difference between target and actual energy mix: {error:.2f}%")

Target energy mix: [ 9. 36. 17.  2. 36.]
Actual energy mix: [15.21589286 67.85714286  4.49839286  0.         12.42857143]
Difference between target and actual energy mix: 76.15%


In [17]:
pmaxs

array([1500.        , 2000.        ,  839.7       ,  563.63636364,
       1392.        ])

In [18]:
pmaxs.sum()

6295.336363636364